In [6]:
import json
 
class Car:
    def __init__(self, car_id, car_model, car_price, is_available=True):
        self.car_id = car_id
        self.car_model = car_model
        self.car_price = car_price
        self.is_available = is_available
 
    def to_dict(self):
        return {
            "car_id": self.car_id,
            "car_model": self.car_model,
            "car_price": self.car_price,
            "is_available": self.is_available
        }
 
    @classmethod
    def from_dict(cls, car_dict):
        return cls(
            car_id=car_dict["car_id"],
            car_model=car_dict["car_model"],
            car_price=car_dict["car_price"],
            is_available=car_dict.get("is_available", True)
        )
 
    def __str__(self):
        availability = "Car available" if self.is_available else "Car not available"
        return f"Car {self.car_id}: {self.car_model} ({availability}) price: {self.car_price} $"
 
 
class Customer:
    def __init__(self, customer_id, customer_name, customer_phone, car_rental_customer=None):
        self.customer_id = customer_id
        self.customer_name = customer_name
        self.customer_phone = customer_phone
        self.car_rental_customer = car_rental_customer or []
 
    def to_dict(self):
        return {
            "customer_id": self.customer_id,
            "customer_name": self.customer_name,
            "customer_phone": self.customer_phone,
            "car_rental_customer": [rental.to_dict() for rental in self.car_rental_customer]
        }
 
    @classmethod
    def from_dict(cls, customer_dict):
        car_rental_customer = [Rental.from_dict(rental_dict) for rental_dict in customer_dict["car_rental_customer"]]
        return cls(
            customer_id=customer_dict["customer_id"],
            customer_name=customer_dict["customer_name"],
            customer_phone=customer_dict["customer_phone"],
            car_rental_customer=car_rental_customer
        )
 
    def __str__(self):
        car_list_customer = ", ".join(rental.car.car_model for rental in self.car_rental_customer)
        return f"Customer {self.customer_id}: {self.customer_name}, phone number: {self.customer_phone}\nRented Cars: {car_list_customer}"
 
 
class Rental:
    def __init__(self, car, customer, days, total_cost=0):
        self.car = car
        self.customer = customer
        self.days = days
        self.total_cost = total_cost
 
    def to_dict(self):
        return {
            "car": self.car.to_dict(),
            "customer": self.customer.to_dict(),
            "days": self.days,
            "total_cost": self.total_cost
        }
 
    @classmethod
    def from_dict(cls, rental_dict):
        car = Car.from_dict(rental_dict["car"])
        customer = Customer.from_dict(rental_dict["customer"])
        return cls(
            car=car,
            customer=customer,
            days=rental_dict["days"],
            total_cost=rental_dict["total_cost"]
        )
 
    def rent_car(self):
        if self.car.is_available:
            self.total_cost = self.car.car_price * self.days
            self.car.is_available = False
            self.customer.car_rental_customer.append(self)
            return True
        else:
            print("Car is not available for rent.")
            return False
 
    def return_car(self):
        self.car.is_available = True
 
 
def save_data_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
 
def load_data_from_json(filename):
    try:
        with open(filename, 'r') as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError:
        return None
 
def display_menu():
    print("\nMenu:")
    print("1. Add a car")
    print("2. Add a customer")
    print("3. Display available cars")
    print("4. Display customer rentals")
    print("5. Reserve a car")
    print("6. Exit")
    
def main():
    cars = []
    customers = []
    rentals = []
 
    while True:
        display_menu()
        choice = input("Enter your choice (1-6): ")
 
        if choice == "1":
            car_id = int(input("Enter car id:"))
            car_model = input("Enter car name and model")
            price_of_car = int(input("Enter price of rent"))
 
            car = Car(car_id, car_model, price_of_car)
 
            cars.append(car)
 
            cars_data = [car.to_dict() for car in cars]
            data_to_save = {"cars": cars_data}#, "customers": customers_data}
            save_data_to_json(data_to_save, "car_data.json")
 
        elif choice == "2":
            customer_id = int(input("Enter customer id:"))
            customer_name = input("Enter customer full name")
            customer_numb = int(input("Enter customer number"))
 
            cust = Customer(customer_id, customer_name, customer_numb)
 
            customers.append(cust)
 
            customers_data = [customer.to_dict() for customer in customers]
            data_to_save = {"customers": customers_data}
            save_data_to_json(data_to_save, "customers_data.json")
 
        elif choice == "3":
            loaded_data = load_data_from_json("car_data.json")
            loaded_cars = [Car.from_dict(car_data) for car_data in loaded_data.get("cars", [])]
            print("Available Cars:")
            for car in loaded_cars:
                print(car)
 
        elif choice == "4":
            loaded_data = load_data_from_json("customers_data.json")
            loaded_customers = [Customer.from_dict(customer_data) for customer_data in loaded_data.get("customers", [])]
            print("Customer Rentals:")
            for customer in loaded_customers:
                print(customer)
 
 
        elif choice == "5":
 
 
            customer_id = int(input("Enter customer id: "))
 
            car_id = int(input("Enter car id to reserve: "))
 
 
            loaded_data_cust = load_data_from_json("customers_data.json")
            loaded_customers = [Customer.from_dict(customer_data) for customer_data in loaded_data_cust.get("customers", [])]
            customer = next((c for c in loaded_customers if c.customer_id == customer_id), None)
 
 
 
            loaded_data_cars = load_data_from_json("car_data.json")
            loaded_cars = [Car.from_dict(car_data) for car_data in loaded_data_cars.get("cars", [])]
            car = next((c for c in loaded_cars if c.car_id == car_id and c.is_available), None)
 
 
 
            if customer and car:
 
                days = int(input("Enter the number of days to reserve: "))
 
                rental = Rental(car, customer, days)
 
                if rental.rent_car():
 
                    rentals.append(rental)
 
                    with open('car_data.json', 'r') as json_file:
                        data = json.load(json_file)
 
 
                    if 'cars' in data:
                        for car in data['cars']:
                            if 'is_available' in car:
                                car['is_available'] = False
 
 
                    with open('car_data.json', 'w') as json_file:
                        json.dump(data, json_file, indent=4)
 
                    print("Car reserved successfully!")
 
                else:
 
                    print("Car is not available for reservation.")
 
 
            else:
 
                print("Customer or car not found.")
 
        elif choice == "6":
            break
 
        else:
            print("Invalid choice. Please enter a number between 1 and 5.")
 
if __name__ == "__main__":
    main()


Menu:
1. Add a car
2. Add a customer
3. Display available cars
4. Display customer rentals
5. Reserve a car
6. Exit
Car reserved successfully!

Menu:
1. Add a car
2. Add a customer
3. Display available cars
4. Display customer rentals
5. Reserve a car
6. Exit
Available Cars:
Car 1234: nissan skyline (Car not available) price: 200 $

Menu:
1. Add a car
2. Add a customer
3. Display available cars
4. Display customer rentals
5. Reserve a car
6. Exit
Available Cars:
Car 1234: nissan skyline (Car not available) price: 200 $

Menu:
1. Add a car
2. Add a customer
3. Display available cars
4. Display customer rentals
5. Reserve a car
6. Exit
